In [1]:
# # Constant viscosity convection, Cartesian domain (benchmark)
#
#
#
# This example solves 2D dimensionless isoviscous thermal convection with a Rayleigh number, for comparison with the [Blankenbach et al. (1989) benchmark](https://academic.oup.com/gji/article/98/1/23/622167).
#
# We set up a v, p, T system in which we will solve for a steady-state T field in response to thermal boundary conditions and then use the steady-state T field to compute a stokes flow in response.

# +tolerace
import os
os.environ["UW_TIMING_ENABLE"] = "1"
import petsc4py
from petsc4py import PETSc


import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import numpy as np
import sympy


In [2]:
# +
##### Set some things
Ra = 1e4 #### Rayleigh number
eta0=1.e23

k = 1.0 #### diffusivity

res= 96 ### x and y res of box

boxLength = 1.0
boxHeight = 1.0
tempMin   = 0.
tempMax   = 1.

viscosity = 1

# +
meshbox = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), cellSize=1.0 / res, regular=False, qdegree=3
)

In [3]:
infile = "/Users/jgra0019/Documents/codes/uw3-dev/TALA-EBA-benchmark/conv_meshes/conv_run12_96"
#infile = "/Users/jgra0019/Documents/codes/uw3-dev/TALA-EBA-benchmark/TALA_meshes/tala_res96_2"

outfig = '/Users/jgra0019/Documents/codes/uw3-dev/local-code/figs/conv12_96.png' 
outfig = None

v_soln = uw.discretisation.MeshVariable("U", meshbox, meshbox.dim, degree=2)
p_soln = uw.discretisation.MeshVariable("P", meshbox, 1, degree=1)
t_soln = uw.discretisation.MeshVariable("T", meshbox, 1, degree=3)
# additional variable for the gradient
dTdZ = uw.discretisation.MeshVariable(r"\partial T/ \partial \Z", 
                                      meshbox, 
                                      1, 
                                      degree = 3) 

v_soln.read_from_vertex_checkpoint(infile + ".U.0.h5", data_name="U")
p_soln.read_from_vertex_checkpoint(infile + ".P.0.h5", data_name="P")
t_soln.read_from_vertex_checkpoint(infile + ".T.0.h5", data_name="T")

x, z = meshbox.X

# projection object to calculate the gradient along Z
dTdZ_calc = uw.systems.Projection(meshbox, dTdZ)
dTdZ_calc.uw_function = t_soln.sym.diff(z)[0]
dTdZ_calc.smoothing = 1.0e-3
dTdZ_calc.petsc_options.delValue("ksp_monitor")

In [4]:
# check the mesh if in a notebook / serial
# allows you to visualise the mesh and the mesh variable
'''FIXME: change this so it's better'''

def plotFig(meshbox, s_field, v_field, s_field_name, save_fname = None, with_arrows = False, cmap = "coolwarm"): 
    """
    s_field - scalar field - corresponds to colors
    v_field - vector field - usually the velocity - 2 components
    """
    if uw.mpi.size == 1:

        import numpy as np
        import pyvista as pv
        import vtk

        pv.global_theme.background = "white"
        pv.global_theme.window_size = [500, 500]
        pv.global_theme.anti_aliasing = None #"ssaa", "msaa", "fxaa", or None
        #pv.global_theme.jupyter_backend = "panel"
        pv.global_theme.smooth_shading = True

        meshbox.vtk("tmp_box_mesh.vtk")
        pvmesh = pv.read("tmp_box_mesh.vtk")

        velocity = np.zeros((meshbox.data.shape[0], 3))
        velocity[:, 0] = uw.function.evaluate(v_field.sym[0], meshbox.data)
        velocity[:, 1] = uw.function.evaluate(v_field.sym[1], meshbox.data)

        #pvmesh.point_data["V"] = velocity / 10

        points = np.zeros((s_field.coords.shape[0], 3))
        points[:, 0] = s_field.coords[:, 0]
        points[:, 1] = s_field.coords[:, 1]

        point_cloud = pv.PolyData(points)

        with meshbox.access():
            point_cloud.point_data[s_field_name] = uw.function.evaluate(s_field.fn, points[:, 0:2])

        # point sources at cell centres
        if meshbox._centroids.shape[0]%4 == 0:
            skip = 4
        elif meshbox._centroids.shape[0]%3 == 0:
            skip = 3
        elif meshbox._centroids.shape[0]%2 == 0:
            skip = 2
        else: 
            skip = 1

        skip = 2
        num_el =  meshbox._centroids[::skip, 0].shape[0]

        cpoints = np.zeros((num_el, 3))
        cpoints[:, 0] = meshbox._centroids[::skip, 0]
        cpoints[:, 1] = meshbox._centroids[::skip, 1]

        cpoint_cloud = pv.PolyData(cpoints)

        # pvstream = pvmesh.streamlines_from_source(
        #     cpoint_cloud,
        #     vectors="V",
        #     integrator_type=2,
        #     integration_direction="forward",
        #     compute_vorticity=False,
        #     max_steps=1000,
        #     surface_streamlines=True,
        # )
 
        pl = pv.Plotter()

        with meshbox.access():
            # point sources at cell centres
            if v_field.coords.shape[0]%20 == 0:
                skip = 20
            elif v_field.coords.shape[0]%4 == 0:
                skip = 4
            elif v_field.coords.shape[0]%3 == 0:
                skip = 3
            elif v_field.coords.shape[0]%2 == 0:
                skip = 2
            else: 
                skip = 1
            
            skip = 2
        
            num_el =  v_field.coords[::skip, 0:2].shape[0]
            arrow_loc = np.zeros((num_el, 3))
            arrow_loc[:, 0:2] = v_field.coords[::skip, 0:2]

            num_el =  v_field.data[::skip, 0].shape[0]
            arrow_length = np.zeros((num_el, 3))
            arrow_length[:, 0] = v_field.data[::skip, 0]
            arrow_length[:, 1] = v_field.data[::skip, 1]

        pl = pv.Plotter()

        #pl.add_mesh(pvmesh,'Gray', 'wireframe')

        pl.add_mesh(
            pvmesh, cmap=cmap, edge_color="Black",
            show_edges=True, use_transparency=False, opacity=0.1,
        )

      
        if with_arrows:
            pl.add_arrows(arrow_loc, arrow_length, mag=0.04, opacity=0.8)
        else:
            pl.add_points(point_cloud, cmap=cmap, point_size=18, opacity=0.8)


        # pl.add_mesh(pvstream, opacity=0.5)
        # pl.add_arrows(arrow_loc2, arrow_length2, mag=1.0e-1)

        # pl.add_points(pdata)

        pl.show(cpos="xy", jupyter_backend = "panel")

        if save_fname is not None:
            #pl.save_graphic(save_fname, dpi = 300)
            pl.image_scale = 3
            pl.screenshot(save_fname) 

        pvmesh.clear_data()
        pvmesh.clear_point_data()
        
        
#plotFig()

In [5]:
# underworld3 function for calculating the rms velocity 
import math

def v_rms(mesh = meshbox, v_solution = v_soln): 
    # v_soln must be a variable of mesh
    v_rms = math.sqrt(uw.maths.Integral(mesh, v_solution.fn.dot(v_solution.fn)).evaluate())
    return v_rms

v_rms()

51.37593294100601

In [6]:
dTdZ_calc.solve() # solve dTdZ

  0 SNES Function norm 0.00682488 
  1 SNES Function norm 9.7883e-06 
  2 SNES Function norm 9.148e-11 
Nonlinear SProj_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2


In [7]:
# function for calculating the surface integral 
def surface_integral(mesh, uw_function, mask_fn):

    calculator = uw.maths.Integral(mesh, uw_function * mask_fn)
    value = calculator.evaluate()

    calculator.fn = mask_fn
    norm = calculator.evaluate()

    integral = value / norm

    return integral

''' set-up surface expressions for calculating Nu number '''
sdev = 0.5*(1/math.sqrt(2*math.log(2)))*(1/res) 
#sdev = 1/res
up_surface_defn_fn = sympy.exp(-((z - 1)**2)/(2*sdev**2)) # at z = 1
lw_surface_defn_fn = sympy.exp(-(z**2)/(2*sdev**2)) # at z = 0

up_int = surface_integral(meshbox, dTdZ.sym[0], up_surface_defn_fn)
lw_int = surface_integral(meshbox, t_soln.sym[0], lw_surface_defn_fn)

Nu = -up_int/lw_int
print(Nu)

3.653199332000905


In [10]:
outfig

In [9]:
plotFig(meshbox, t_soln, v_soln, "T", save_fname = outfig, with_arrows = False)

BokehModel(combine_events=True, render_bundle={'docs_json': {'50c8247d-f863-4c44-a2d0-f7d97ce12d57': {'defs': …